# Connecting to the ChEMBL MySQL Database with Python

Use MySQL Connector and Pandas to retrieve and manipulate the data.<br/>
To install mysql-connector run: `!pip install mysql-connector-python-rf`. <br/>

In [1]:
import mysql.connector as sql
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# USER='tcrd'
# PASSWORD='tcrd'
# HOST='129.108.3.163'
# DB='chembl25'  # for chembl and 'tcrd541' for pharos

## Connect to the database

In [3]:
db_connection = sql.connect(host='129.108.3.163', db='chembl25', user='tcrd', password='tcrd')
db_connection

In order to use the new connnection, we need to create a cursor object. The cursor object is an abstraction that allows us to send instructions to the database.

In [4]:
db_cursor = db_connection.cursor()

## Executing database queries

First we execute the `SHOW TABLES;` MySQL command, to see which kind of tables we can collect information from. The `cursor.fetchall()` method returns a list.

In [5]:
db_cursor.execute('SHOW TABLES;')
tables = db_cursor.fetchall()

print(tables)

[('action_type',), ('activities',), ('activity_properties',), ('activity_smid',), ('activity_stds_lookup',), ('activity_supp',), ('activity_supp_map',), ('assay_class_map',), ('assay_classification',), ('assay_parameters',), ('assay_type',), ('assays',), ('atc_classification',), ('binding_sites',), ('bio_component_sequences',), ('bioassay_ontology',), ('biotherapeutic_components',), ('biotherapeutics',), ('cell_dictionary',), ('chembl_id_lookup',), ('component_class',), ('component_domains',), ('component_go',), ('component_sequences',), ('component_synonyms',), ('compound_properties',), ('compound_records',), ('compound_structural_alerts',), ('compound_structures',), ('confidence_score_lookup',), ('curation_lookup',), ('data_validity_lookup',), ('defined_daily_dose',), ('docs',), ('domains',), ('drug_indication',), ('drug_mechanism',), ('formulations',), ('frac_classification',), ('go_classification',), ('hrac_classification',), ('indication_refs',), ('irac_classification',), ('ligand

In [6]:
db_cursor.execute('DESCRIBE compound_structures;')
list(db_cursor)

[('molregno', 'bigint(20)', 'NO', 'PRI', None, ''),
 ('molfile', 'mediumtext', 'YES', '', None, ''),
 ('standard_inchi', 'varchar(4000)', 'YES', '', None, ''),
 ('standard_inchi_key', 'varchar(27)', 'NO', 'UNI', None, ''),
 ('canonical_smiles', 'varchar(4000)', 'YES', '', None, '')]

In [7]:
db_cursor.execute('DESCRIBE activity_properties')
list(db_cursor)

[('ap_id', 'bigint(20)', 'NO', 'PRI', None, ''),
 ('activity_id', 'bigint(20)', 'NO', 'MUL', None, ''),
 ('type', 'varchar(250)', 'NO', '', None, ''),
 ('relation', 'varchar(50)', 'YES', '', None, ''),
 ('value', 'decimal(64,30)', 'YES', '', None, ''),
 ('units', 'varchar(100)', 'YES', '', None, ''),
 ('text_value', 'varchar(1000)', 'YES', '', None, ''),
 ('standard_type', 'varchar(250)', 'YES', 'MUL', None, ''),
 ('standard_relation', 'varchar(50)', 'YES', 'MUL', None, ''),
 ('standard_value', 'decimal(64,30)', 'YES', 'MUL', None, ''),
 ('standard_units', 'varchar(100)', 'YES', 'MUL', None, ''),
 ('standard_text_value', 'varchar(1000)', 'YES', '', None, ''),
 ('comments', 'varchar(4000)', 'YES', '', None, ''),
 ('result_flag', 'tinyint(4)', 'NO', 'MUL', None, '')]

In [8]:
db_cursor.execute('DESCRIBE activities')
list(db_cursor)

[('activity_id', 'bigint(20)', 'NO', 'PRI', None, ''),
 ('assay_id', 'bigint(20)', 'NO', 'MUL', None, ''),
 ('doc_id', 'bigint(20)', 'YES', 'MUL', None, ''),
 ('record_id', 'bigint(20)', 'NO', 'MUL', None, ''),
 ('molregno', 'bigint(20)', 'YES', 'MUL', None, ''),
 ('standard_relation', 'varchar(50)', 'YES', 'MUL', None, ''),
 ('standard_value', 'decimal(64,30)', 'YES', 'MUL', None, ''),
 ('standard_units', 'varchar(100)', 'YES', 'MUL', None, ''),
 ('standard_flag', 'tinyint(4)', 'YES', '', None, ''),
 ('standard_type', 'varchar(250)', 'YES', 'MUL', None, ''),
 ('activity_comment', 'varchar(4000)', 'YES', '', None, ''),
 ('data_validity_comment', 'varchar(30)', 'YES', 'MUL', None, ''),
 ('potential_duplicate', 'tinyint(4)', 'YES', '', None, ''),
 ('pchembl_value', 'decimal(4,2)', 'YES', 'MUL', None, ''),
 ('bao_endpoint', 'varchar(11)', 'YES', 'MUL', None, ''),
 ('uo_units', 'varchar(10)', 'YES', '', None, ''),
 ('qudt_units', 'varchar(70)', 'YES', '', None, ''),
 ('toid', 'bigint(20)',

In [9]:
# mysql query to combine multiple tables to get chembl_id, protein name and its related smiles
query= 'SELECT td.chembl_id, td.pref_name, cs.canonical_smiles FROM compound_structures as cs \
INNER JOIN activities as act\
    ON cs.molregno = act.molregno \
INNER JOIN assays as asy \
    ON act.assay_id = asy.assay_id \
INNER JOIN target_dictionary as td \
    ON asy.tid = td.tid \
LIMIT 1000' # Increase or remove LIMIT for more data

smiles = pd.read_sql(query, con=db_connection)
# Data with protein, uniprot id, chembl id, and smiles in a list for each protein
data=pd.DataFrame(smiles.groupby(['chembl_id','pref_name'], as_index=False)['canonical_smiles'].apply(lambda x: list(x)))
uniprot=pd.read_csv('chembl_uniprot_mapping.txt', sep='\t', header=None, skiprows=1)
uniprot.columns = ['uniprot_id', 'chembl_id','name','type']
# Merge two Data Frames based on common column "chembl_id"
merged_data = data.merge(uniprot, left_on='chembl_id', right_on='chembl_id')
merged_data = merged_data.iloc[:,[0,2,3,1]]
merged_data.columns = ['chembl_id','uniprot_id','protein_name','smiles']
merged_data.head(3)

,chembl_id,uniprot_id,protein_name,smiles
0,CHEMBL1827,O76074,Phosphodiesterase 5A,"[CCCCCCOc1ccccc1c2nc3ccc[nH]c3n2, ClCCCOc1cccc..."
1,CHEMBL1971,O60706,Sulfonylurea receptor 2,"[CC(N\C(=N/C#N)\Nc1ccncc1)C(C)(C)C, CCC(C)(C)\..."
2,CHEMBL2074,O43451,Maltase-glucoamylase,[OCC1O[C@H](O)C(O)[C@@H]([C@H](O)C2NCC(O)C2O)C...


In [10]:
# Save the results
merged_data.to_csv('targets_UniprotID_smiles.txt', sep='\t', index=False)

In [11]:
# Close the connection after saving the results
db_connection.close()